In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install tqdm transformers deepspeed bitsandbytes langchain sentencepiece protobuf peft einops

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:
def llm_generate(input_txt):
    sequences = pipeline(
            input_txt,
            max_length=512,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
    )
    return sequences[0]['generated_text']
print(llm_generate("Why is earth round?"))

In [3]:
template = \
"""Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
{summary} 

Article: 
{article} 

Explain your reasoning step by step:
"""

doc1 = "Mr Katter said the Government believes Mr Gordon would quit after he was recently accused of domestic violence."
sum1 = "Mr Katter said he would quit after he was accused of domestic violence."

doc2 = "Barcelona club president Josep Maria Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're `very happy' with him."
sum2 = "Josep Maria Bartomeu says the La Liga leaders are very happy with him."

doc3 = "Goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one metre in length."
sum3 = "Goldfish are being caught weighing up to 8kg and one metre in length."

doc4 = "Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley ...Townsend hit back at Merson on Twitter after scoring for England against Italy."
sum4 = "Paul Merson had another dig at andros townsend after scoring for England against Italy."

doc5 = "Catania, Italy (CNN)The boat that sank in the Mediterranean over the weekend with hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid, a U.N. official said. Carlotta Asami, a spokeswoman for the United Nations' refugee agency, made the comments to CNN early Tuesday after she and two of her colleagues had spoken to multiple survivors from the disaster who arrived in Catania, Italy."
sum5 = "hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid."

In [4]:
print(llm_generate(template.format(article=doc1, summary=sum1)))

"""
Original:
1. The summary states that Mr Katter would quit after being accused of domestic violence. 
2. However, the article states that the Government believes Mr Gordon would quit after being recently accused of domestic violence. 
3. The inconsistency is due to the use of the word'recently'. The article implies that the Government believes Mr Gordon would quit after being recently accused, while the summary states that Mr Katter would quit after being accused. 
4. To maintain consistency, the summary should use the same word as the article,'recently'.

###Corrected:
1. The summary states that Mr Katter would quit after being accused of domestic violence. 
2. However, the article states that the Government believes Mr Gordon would quit after being recently accused of domestic violence. 
3. The inconsistency is due to the error that the article implies that the Government believes **Mr Gordon** would quit, while the summary states that **Mr Katter** would quit. 
"""

Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Mr Katter said he would quit after he was accused of domestic violence. 

Article: 
Mr Katter said the Government believes Mr Gordon would quit after he was recently accused of domestic violence. 

Explain your reasoning step by step:
1. The summary states that Mr Katter would quit after being accused of domestic violence. 
2. However, the article states that the Government believes Mr Gordon would quit after being recently accused of domestic violence. 
3. The inconsistency is due to the use of the word'recently'. The article implies that the Government believes Mr Gordon would quit after being recently accused, while the summary states that Mr Katter would quit after being accused. 
4. To maintain consistency, the summary should use the same word as the article,'recently'.


In [5]:
print(llm_generate(template.format(article=doc2, summary=sum2)))

"""
Original:
1. The summary claims that Bartomeu is happy with himself, which contradicts the article's claim that the La Liga leaders are very happy with him. 
2. The article provides evidence for the claim that the La Liga leaders are very happy with him, while the summary does not. 
3. Therefore, the summary is not consistent with the article.

###Corrected:
1. The summary claims that Bartomeu says the La Liga leaders are very happy with him.
2. The article provides evidence for the claim that the La Liga leaders have no plans to replace Luis Enrique and they're `very happy' with him. 
3. Therefore, the summary is not consistent with article on that the summary implies the La Liga leaders are very happy with **Bartomeu**, but the article implies the La Liga leaders are very happy with **Luis Enrique**.
"""

Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Josep Maria Bartomeu says the La Liga leaders are very happy with him. 

Article: 
Barcelona club president Josep Maria Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're `very happy' with him. 

Explain your reasoning step by step:
1. The summary claims that Bartomeu is happy with himself, which contradicts the article's claim that the La Liga leaders are very happy with him. 
2. The article provides evidence for the claim that the La Liga leaders are very happy with him, while the summary does not. 
3. Therefore, the summary is not consistent with the article.


In [6]:
print(llm_generate(template.format(article=doc3, summary=sum3)))

"""
Original:
1. The article states that goldfish can weigh up to 8kg, while the summary states that koi carp can weigh up to 8kg. 
2. The article also states that koi carp can reach a length of 1 metre, while the summary states that goldfish can reach a length of up to 8cm. 
3. Therefore, the summary is not consistent with the article.

###Corrected:
1. The article states that **goldfish** can weigh up to 8kg, while the summary states that **koi carp** can weigh up to 8kg. 
2. Therefore, the summary is not consistent with the article.
"""

Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Goldfish are being caught weighing up to 8kg and one metre in length. 

Article: 
Goldfish are being caught weighing up to 2kg and koi carp up to 8kg and one metre in length. 

Explain your reasoning step by step:
1. The article states that goldfish can weigh up to 8kg, while the summary states that koi carp can weigh up to 8kg. 
2. The article also states that koi carp can reach a length of 1 metre, while the summary states that goldfish can reach a length of up to 8cm. 
3. Therefore, the summary is not consistent with the article.


In [7]:
print(llm_generate(template.format(article=doc4, summary=sum4)))

"""
Original
1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.
2. However, the article mentions that Andros Townsend was not even in the match against Italy.
3. Therefore, the summary is not consistent with the article.

###Corrected:
1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.
2. However, the article mentions that Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley.
3. Therefore, the summary is not consistent with the article on the time of event where the summary states **after scoring for England against Italy**, while the article states **after his appearance for Tottenham against Burnley**.
"""

Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
Paul Merson had another dig at andros townsend after scoring for England against Italy. 

Article: 
Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley ...Townsend hit back at Merson on Twitter after scoring for England against Italy. 

Explain your reasoning step by step:
1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.
2. However, the article mentions that Andros Townsend was not even in the match against Italy.
3. Therefore, the summary is not consistent with the article.


In [8]:
print(llm_generate(template.format(article=doc5, summary=sum5)))

"""
Original
Unable to produce reasonning...

###Corrected: 
1. The summary states hundreds of migrants on board may have capsized.
2. The article mentions the boat with hundreds of migrants on board may have capsized.
3. The summary is not consistent article because the summary says **migrants** may have capsized, while the article states **boat** may have capsized.
"""

Note that consistency means all information in the summary is supported by the article. 
It's known that the following summary is not consistent with the article. 
Find out the why.

Summary: 
hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid. 

Article: 
Catania, Italy (CNN)The boat that sank in the Mediterranean over the weekend with hundreds of migrants on board may have capsized after being touched or swamped by a cargo ship that came to its aid, a U.N. official said. Carlotta Asami, a spokeswoman for the United Nations' refugee agency, made the comments to CNN early Tuesday after she and two of her colleagues had spoken to multiple survivors from the disaster who arrived in Catania, Italy. 

Explain your reasoning step by step:
1. The article mentions a cargo ship that came to the aid of the migrants. 2. The cargo ship may have touched or swamped the migrants. 3. The cargo ship may have capsized. 4. The cargo ship m

'\nOriginal\n1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.\n2. However, the article mentions that Andros Townsend was not even in the match against Italy.\n3. Therefore, the summary is not consistent with the article.\n\nCorrected (Hallucination)\n1. The summary states that Paul Merson had another dig at Andros Townsend after scoring for England against Italy.\n2. However, the article mentions that Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley.\n3. Therefore, the summary is not consistent with the article on the time of event where the summary states **after scoring for England against Italy**, while the article states **after his appearance for Tottenham against Burnley**.\n'